In [1]:
# Libraries
import open3d as o3d
import viser
import torch
import numpy as np
import cv2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#@title Importing the Clusters with It's radiances
slices = torch.load("/workspace/FruitProposal/attachment/RadianceCloud/slice_points.pt")
print(slices.keys())

planes  = slices['plane']
points  = slices['points']
rgbs    = slices['rgb']
centers = slices['slice_center']
corners = slices['slice_corner']

print(planes.shape, points.shape, rgbs.shape, centers.shape, corners.shape)
print(planes[0], points[0], rgbs[0], centers[0], corners[0])

dict_keys(['points', 'plane', 'rgb', 'slice_corner', 'slice_center'])
(49,) (49,) (49,) (49, 3) (49, 4, 3)
[] [] [] [ 0.27118844  0.2108236  -0.0116659 ] [[ 0.24521568  0.18485084 -0.0116659 ]
 [ 0.2971612   0.18485084 -0.0116659 ]
 [ 0.24521568  0.23679635 -0.0116659 ]
 [ 0.2971612   0.23679635 -0.0116659 ]]


In [3]:
# Forzamos z=1
_planes = [plane.copy() for plane in planes]
for plane in _planes:
    plane[:,2] = 1.0
_centers  = centers.copy();  _centers[:,2]  = 1.0

_corners  = [c.copy() for c in corners]
for c in _corners:
    c[:,2] = 1.0

In [4]:
# 1) Calcular R solo una vez (misma orientación para todas)
z_cam    = np.array([0,0,-1], dtype=np.float32)
e        = _corners[0][0] - _corners[0][1]
e_norm   = np.linalg.norm(e)
y_cam    = e / e_norm
x_cam    = np.cross(y_cam, z_cam); x_cam /= np.linalg.norm(x_cam)
y_cam    = np.cross(z_cam, x_cam); y_cam /= np.linalg.norm(y_cam)
R        = np.column_stack((x_cam, y_cam, z_cam))
print("R\n", R)


R
 [[-0. -1.  0.]
 [-1.  0.  0.]
 [-0.  0. -1.]]


In [5]:
# 2) Intrínsecos
W = H = 512
cx = cy = W/2
fxy      = W
intrinsic = np.array([
    [fxy,   0, cx],
    [  0, fxy, cy],
    [  0,   0,  1]
], dtype=np.float32)
print("Intrinsic\n", intrinsic)

Intrinsic
 [[512.   0. 256.]
 [  0. 512. 256.]
 [  0.   0.   1.]]


In [ ]:
# 3) Bucle por cada slice
for k, (plane_k, rgb_k, center_k) in enumerate(
        zip(_planes, rgbs, _centers)):

    # Renderizar slice
    void_image = np.zeros((H, W, 3), dtype=np.uint8)

    # Posición inicial de la cámara sobre el centro del slice
    C = center_k + np.array([0,0,e_norm], dtype=np.float32)
    t = -R @ C

    for pt, color in zip(plane_k, rgb_k):

        Xc = R @ pt.T + t
        uvw = intrinsic @ Xc
        u, v = uvw[0]/uvw[2], uvw[1]/uvw[2]

        ui, vi = int(round(u)), int(round(v))
        if 0 <= ui < W and 0 <= vi < H:
            void_image[vi, ui] = color * 255

    # 3.4) Dilatación y guardado
    kernel = np.ones((3,3), np.uint8)
    out    = cv2.dilate(cv2.cvtColor(void_image, cv2.COLOR_RGB2BGR), kernel, iterations=5)
    cv2.imwrite(f"/workspace/FruitProposal/attachment/RadianceCloud/images/slice_{k}.png", out)
    out_hsv    = cv2.dilate(cv2.cvtColor(void_image, cv2.COLOR_RGB2HSV), kernel, iterations=5)
    cv2.imwrite(f"/workspace/FruitProposal/attachment/RadianceCloud/images/slice_hsv_{k}.png", out_hsv)

Point [0.2578007  0.18962431 1.        ] -> Camera (0.021199285984039307, 0.013387739658355713, 0.05194544792175293) -> Image (464.95062255859375, 387.9561767578125)
Point [0.2578007  0.18962431 1.        ] -> Image (465, 388)
Point [0.2784992  0.21941476 1.        ] -> Camera (-0.00859116017818451, -0.007310748100280762, 0.05194544792175293) -> Image (171.32127380371094, 183.941650390625)
Point [0.2784992  0.21941476 1.        ] -> Image (171, 184)
Point [0.2792523 0.218939  1.       ] -> Camera (-0.008115410804748535, -0.008063852787017822, 0.05194544792175293) -> Image (176.010498046875, 176.5186767578125)
Point [0.2792523 0.218939  1.       ] -> Image (176, 177)
Point [0.2777112  0.21965742 1.        ] -> Camera (-0.008833825588226318, -0.006522774696350098, 0.05194544792175293) -> Image (168.929443359375, 191.70831298828125)
Point [0.2777112  0.21965742 1.        ] -> Image (169, 192)
Point [0.2591268  0.21109611 1.        ] -> Camera (-0.00027251243591308594, 0.012061625719070435